In [6]:
!wget https://www.dropbox.com/s/ie7h1hob7uhuf5v/archive.zip?dl=0

--2022-06-05 09:30:39--  https://www.dropbox.com/s/ie7h1hob7uhuf5v/archive.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6022:18::a27d:4212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/ie7h1hob7uhuf5v/archive.zip [following]
--2022-06-05 09:30:39--  https://www.dropbox.com/s/raw/ie7h1hob7uhuf5v/archive.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc409824e7d24655075fcf1fb343.dl.dropboxusercontent.com/cd/0/inline/BmmstoqeEoUmYvM7ZvrBSUYBBvhRLv3KrQH0oBVFozo0F-6_Lu2f1LE22F8h9PqUsJHhjCyMltFuWQZ1-A_PAbFI5c3zHEbzMB4sVP_KdNK9ySPNqxA0CfuJB2ZDQNnrHWSnn7wXLYh8yyYORgJfB2g2ZwaMT6UL8Hs_VMwteG1wHg/file# [following]
--2022-06-05 09:30:39--  https://uc409824e7d24655075fcf1fb343.dl.dropboxusercontent.com/cd/0/inline/BmmstoqeEoUmYvM7ZvrBSUYBBvhRLv3KrQH0oBVFozo0F-6_Lu2f1LE22F8h9PqUs

In [7]:
!unzip archive.zip?dl=0

Archive:  archive.zip?dl=0
  inflating: dataset/cats/cat.1.jpg  
  inflating: dataset/cats/cat.10.jpg  
  inflating: dataset/cats/cat.100.jpg  
  inflating: dataset/cats/cat.101.jpg  
  inflating: dataset/cats/cat.102.jpg  
  inflating: dataset/cats/cat.103.jpg  
  inflating: dataset/cats/cat.104.jpg  
  inflating: dataset/cats/cat.105.jpg  
  inflating: dataset/cats/cat.106.jpg  
  inflating: dataset/cats/cat.107.jpg  
  inflating: dataset/cats/cat.108.jpg  
  inflating: dataset/cats/cat.109.jpg  
  inflating: dataset/cats/cat.11.jpg  
  inflating: dataset/cats/cat.110.jpg  
  inflating: dataset/cats/cat.111.jpg  
  inflating: dataset/cats/cat.112.jpg  
  inflating: dataset/cats/cat.113.jpg  
  inflating: dataset/cats/cat.114.jpg  
  inflating: dataset/cats/cat.115.jpg  
  inflating: dataset/cats/cat.116.jpg  
  inflating: dataset/cats/cat.117.jpg  
  inflating: dataset/cats/cat.118.jpg  
  inflating: dataset/cats/cat.119.jpg  
  inflating: dataset/cats/cat.12.jpg  
  inflating: datas

In [8]:
!ls

'archive.zip?dl=0'   dataset   sample_data


In [18]:
import os
folders = os.listdir("dataset")
folders.remove("dataset")
print(folders)

['cats', 'dogs', 'humans', 'horses']


In [17]:
for f in folders:
  path = "dataset/"+f
  print(f+" "+str(len(os.listdir(path))))

cats 202
dogs 202
humans 202
horses 202


In [19]:
image_data = []
label = []

label_dict = {
    "cats" : 0,
    "dogs" : 1,
    "humans":2,
    "horses":3
}


In [21]:
from keras.preprocessing import image
for ix in folders:
  path = os.path.join("dataset",ix)
  for im in os.listdir(path):
    img = image.load_img(os.path.join(path,im),target_size = ((224,224,3)))
    image_array = image.img_to_array(img)
    image_data.append(image_array)
    label.append(label_dict[ix])

In [22]:
print(len(image_data),len(label))

808 808


In [26]:
import random
combined = list(zip(image_data,label))
random.shuffle(combined)

image_data[:],label[:] = zip(*combined)

In [27]:
print(label[1:10])

[0, 3, 3, 0, 2, 1, 0, 3, 2]


In [28]:
import numpy as np
x_train= np.array(image_data)
y_train = np.array(label)

print(x_train.shape,y_train.shape)


(808, 224, 224, 3) (808,)


In [30]:
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train)
print(x_train.shape,y_train.shape)


(808, 224, 224, 3) (808, 4)


In [39]:
#create the model ResNet50
from keras.applications.resnet import ResNet50
from keras.layers import *
from keras.optimizers import adam_experimental
from keras.models import Model


In [80]:
model = ResNet50(include_top=False,weights = "imagenet",input_shape=(224,224,3))

In [81]:
print(model.summary())

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [84]:

av1 = GlobalAveragePooling2D()(model.output)
fc1 = Dense(256,activation="relu")(av1)
d1 = Dropout(0.5)(fc1)
fc2 = Dense(4,activation="softmax")(d1)


model_new = Model(inputs = model.input,outputs = fc2)
model_new.summary()

Model: "model_18"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [88]:
from keras import optimizers
from keras.optimizer_v2.adam import Adam
adam = Adam(lr = 0.0003)
model_new.compile(loss = "categorical_crossentropy",optimizer=adam,metrics = "accuracy")

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [87]:
for ix in range(len(model_new.layers)):
  print(ix,model_new.layers[ix])
  

0 <keras.engine.input_layer.InputLayer object at 0x7f9cf68234d0>
1 <keras.layers.convolutional.ZeroPadding2D object at 0x7f9c645d6cd0>
2 <keras.layers.convolutional.Conv2D object at 0x7f9c645d6f10>
3 <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f9c6462bd10>
4 <keras.layers.core.activation.Activation object at 0x7f9c64630a90>
5 <keras.layers.convolutional.ZeroPadding2D object at 0x7f9c6464cd10>
6 <keras.layers.pooling.MaxPooling2D object at 0x7f9c646c6350>
7 <keras.layers.convolutional.Conv2D object at 0x7f9c64679310>
8 <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f9c646fa0d0>
9 <keras.layers.core.activation.Activation object at 0x7f9c64668ed0>
10 <keras.layers.convolutional.Conv2D object at 0x7f9c646ac050>
11 <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f9c645d9950>
12 <keras.layers.core.activation.Activation object at 0x7f9c645e1050>
13 <keras.layers.convolutional.Conv2D object at 

In [91]:
for ix in range(169):
  model_new.layers[ix].trainable = False

model_new.compile(loss = "categorical_crossentropy",optimizer=adam,metrics = "accuracy")
model_new.summary()

Model: "model_18"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [93]:
hist = model_new.fit(x_train,y_train,
                 shuffle = True,
                 batch_size = 16,
                 epochs = 5,
                 validation_split = 0.20)

Epoch 1/5
41/41 [==============================] - 20s 117ms/step - loss: 0.5407 - accuracy: 0.8019 - val_loss: 0.2221 - val_accuracy: 0.9383
Epoch 2/5
41/41 [==============================] - 3s 70ms/step - loss: 0.0953 - accuracy: 0.9675 - val_loss: 0.0791 - val_accuracy: 0.9630
Epoch 3/5
41/41 [==============================] - 3s 70ms/step - loss: 0.0506 - accuracy: 0.9845 - val_loss: 0.0696 - val_accuracy: 0.9630
Epoch 4/5
41/41 [==============================] - 3s 70ms/step - loss: 0.0287 - accuracy: 0.9954 - val_loss: 0.0360 - val_accuracy: 0.9877
Epoch 5/5
41/41 [==============================] - 3s 71ms/step - loss: 0.0228 - accuracy: 0.9938 - val_loss: 0.0472 - val_accuracy: 0.9815
